# Counting the Bodies

My world recently suffered from a massive raid. From the logs, 43 villagers died. I also want to get a count of the Iron Golems who gave their lives to the cause.

In [1]:
import json
from os import environ
from pathlib import Path
from typing import Any

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
worlds_folder = Path(environ["SAVES_PATH"])

In [3]:
def find_iron_golem(path: Path) -> list[tuple[int, ...]]:
    """Find and return all iron golems in a given
    entities file

    Parameters
    ----------
    path : Path
        The path to the entities file

    Returns
    -------
    list of tuples
        The locations of all found golems
    """
    golems: list[tuple[int, ...]] = []
    for chunk in region.RegionFile(path).iter_chunks():
        for entity in chunk["Entities"]:
            if entity["id"].value not in ("minecraft:iron_golem",):
                continue
            golems.append(
                tuple(int(v.value) for i, v in enumerate(entity["Pos"]) if i != 1)
            )
    return golems

## Before the Battle

Investigating a backup from before the battle

In [4]:
entities_folder = worlds_folder / "Reference" / "entities"
golems = []
for filename in ("r.-4.-8.mca", "r.-3.-8.mca"):
    golems.extend(find_iron_golem(entities_folder / filename))

golems = [loc for loc in golems if -1568 < loc[0] < -1393 and -3859 < loc[-1] < -3646]
print(f"Found {len(golems)} golems")

Found 13 golems


## After the Battle

In [5]:
entities_folder = worlds_folder / "Esha Ness" / "entities"
golems = []
for filename in ("r.-4.-8.mca", "r.-3.-8.mca"):
    golems.extend(find_iron_golem(entities_folder / filename))

golems = [loc for loc in golems if -1568 < loc[0] < -1393 and -3859 < loc[-1] < -3646]
print(f"Found {len(golems)} golems")

Found 5 golems


RIP, my iron friends.

## Bonus: Did My Tamed Pillagers Get Into The Fight?

They've got no weapons and shouldn't have been part of the raid.

In [6]:
def get_name(entity: dict[str, Any]) -> str:
    """Return the name (or identifier) of an entity

    Parameters
    ----------
    entity: dict
        The entity of interest

    Returns
    -------
    str
        The name and ID (or just the ID) of the entity
    """
    identifier = entity["id"]

    if "CustomName" not in entity.keys():
        return str(identifier)
    custom_name = json.loads(entity["CustomName"].value)
    try:
        name = custom_name["text"]  # 1.20.1-
    except TypeError:
        name = custom_name  # 1.20.3+
    return f"{name} ({identifier})"

In [7]:
def find_illagers(path: Path) -> list[tuple[str, tuple[int, ...]]]:
    """Find and return all named villagers
    in a given entities file

    Parameters
    ----------
    path : Path
        The path to the entities file

    Returns
    -------
    list of dict
        The attributes of all found villagers
    """
    illagers = []
    for chunk in region.RegionFile(path).iter_chunks():
        for entity in chunk["Entities"]:
            if entity["id"].value not in (
                "minecraft:pillager",
                "minecraft:vindicator",
                "minecraft:witch",
                "minecraft:ravager",
                "minecraft:evoker",
            ):
                continue
            illagers.append(
                (get_name(entity), tuple(int(v.value) for v in entity["Pos"]))
            )
    return illagers

In [8]:
entities_folder = worlds_folder / "Reference" / "entities"
illagers = []
for filename in ("r.-4.-8.mca", "r.-3.-8.mca"):
    illagers.extend(find_illagers(entities_folder / filename))

illagers = [
    (illager, loc)
    for illager, loc in illagers
    if -1632 < loc[0] < -1362 and -3900 < loc[-1] < -3646
]
for illager, loc in illagers:
    print(f"{illager} at {loc}")

Harald (minecraft:pillager) at (-1499, 104, -3897)
Ragnvald (minecraft:pillager) at (-1474, 101, -3880)


In [9]:
entities_folder = worlds_folder / "Esha Ness" / "entities"
illagers = []
for filename in ("r.-4.-8.mca", "r.-3.-8.mca"):
    illagers.extend(find_illagers(entities_folder / filename))

illagers = [
    (illager, loc)
    for illager, loc in illagers
    if -1632 < loc[0] < -1362 and -3900 < loc[-1] < -3646
]
for illager, loc in illagers:
    print(f"{illager} at {loc}")

minecraft:pillager at (-1575, 62, -3701)
Harald (minecraft:pillager) at (-1500, 104, -3898)
Ragnvald (minecraft:pillager) at (-1475, 101, -3880)


Well what do we have here?? The mysterious missing pillager from the last wave that we let timeout?